# Test for Calculating Room Acoustic Parameter
- using module 'pyRoomAcoustic'


In [1]:
'''
# 라이브러리 정의
 - 시스템 라이브러리
 - 오디오 관련 라이브러리
 - 연산 및 Plot 관련 라이브러리 
 - 샤용자 라이브러리
'''

# Import Systems 
import struct
import io
import os
import sys
import time
import platform

# Import Audio
import pyaudio
import librosa
import soundfile

import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

# User Libraries
import pyOssWavfile
import pyRoomAcoustic as room
import pyOssDebug as dbg



In [2]:
#
str_os_system = platform.system()
print(str_os_system)
print(os.name)
print(sys.platform)

Windows
nt
win32


In [3]:
# Path of impulsefiles folder
# impulse_dir = os.path.join(os.getcwd(), 'impulsefiles')
impulse_dir = os.path.join(os.getcwd(), 'ju_impulse')

# Define wave file name text to open
# Path + wave file name

# IMPULSE WAVE FILE
# wav_fname_impulse = "TyndallBruceMonument.wav"
# wav_fname_impulse = "ElvedenHallMarbleHall.wav'"
# wav_fname_impulse = "EmptyApartmentBedroom.wav"
# wav_fname_impulse = "FalklandPalaceRoyalTennisCourt.wav"
# wav_fname_impulse = "InsidePiano.wav"
# wav_fname_impulse = "MaesHowe.wav"
# wav_fname_impulse = "SportsCentreUniversityOfYork.wav"
# wav_fname_impulse = "StairwayUniversityOfYork.wav"
# wav_fname_impulse = "StAndrewsChurch.wav"
# wav_fname_impulse = "mh3_000_ortf_48k.wav"

wav_fname_impulse = "Impulse_201125_JU_SoundLab_Ori_mono_48k_32i.wav"
# wav_fname_impulse = "Impulse_201125_JU_SoundLab_Aram_mono_48k_32i.wav"
# wav_fname_impulse = "Impulse_201125_JU_SoundLab_Sejong_mono_48k_32i.wav"
# wav_fname_impulse = "Impulse_201125_JU_SoundLab_Daegu_mono_48k_32i.wav"

impulse_fname = os.path.join(impulse_dir, wav_fname_impulse)

#print(impulse_fname)

In [4]:
# Read .wav file using pyOssWavfile.read() & Extract .wav file header information

impulse_fmt_chunk = pyOssWavfile.read_format(impulse_fname)
impulse_struct_fmt = pyOssWavfile.extractWavFmtChunk(impulse_fmt_chunk)

if impulse_struct_fmt.bitdepth == 24:        # Bitdepth가 24bit일 때는 pysoundfile을 이용하여 wav data를 불러옴
    impulse_data, impulse_struct_fmt.fs = soundfile.read(impulse_fname)
else:                                    # 그 외에는 scipy.io.wavfile을 수정한 함수 pyOssWavfile을 이용하여 wav data를 불러옴
    impulse_fmt_chunk, impulse_data, impulse_length = pyOssWavfile.read(impulse_fname)

impulse_time = impulse_data.shape[0] / impulse_struct_fmt.fs

print("Audio Format =", impulse_struct_fmt.format)
print("Number of Channel =", impulse_struct_fmt.ch)
print("Sampling Frequency =", impulse_struct_fmt.fs)
print("Byte Rate =", impulse_struct_fmt.byterate)       # 일종의 Checksum 
print("Block Align =", impulse_struct_fmt.blockalign)
print("Bits per Sample =", impulse_struct_fmt.bitdepth)
print("Time =", impulse_time, "sec")
print("Length = ", impulse_data.shape[0])

# Plot loaded wave file
# dbg.dPlotAudio(impulse_struct_fmt.fs, impulse_data[:,0], impulse_fname, "Left Channel", "Time(sec)", "Amplitude")
# dbg.dPlotAudio(ossWav_fs, np.int16(ossWav_data[:,0] * 32767), audio_fname, "Left Channel", "Time(sec)", "Amplitude")
dbg.dPlotAudio(impulse_struct_fmt.fs, impulse_data, impulse_fname, "Left Channel", "Time(sec)", "Amplitude")



Audio Format = 1
Number of Channel = 1
Sampling Frequency = 48000
Byte Rate = 96000
Block Align = 2
Bits per Sample = 16
Time = 5.461333333333333 sec
Length =  262144


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
# Calculate normalized decay curve
impulse_decaycurve = room.decayCurve(impulse_data[:,0], impulse_time, impulse_struct_fmt.fs)
dbg.dPlotAudio(impulse_struct_fmt.fs, impulse_decaycurve, impulse_fname, "Normalized Decay Curve", "Time(sec)", "Amplitude")

# Calculate RT60 (-5 ~ -65dB)
impulse_rt60, impulse_rt60nonLin = room.RT60(impulse_decaycurve, impulse_struct_fmt.fs) 
print("impulse_RT60=", impulse_rt60, "impulse_RT60 nonLin=", impulse_rt60nonLin)            # for Debug

# Calculate EDT(T10 x 6) & T10(0 ~ -10dB) 
impulse_EDT, impulse_EDTnonLin = room.EDT(impulse_decaycurve, impulse_struct_fmt.fs)
print("impulse_EDT=", impulse_EDT, "impulse_EDT nonLin=", impulse_EDTnonLin)            # for Debug
print("impulse_T10=", impulse_EDT/6, "impulse_T10 nonLin=EDT=", impulse_EDTnonLin)      # for Debug

# Calculate T20 (-5 ~ -25dB)
impulse_t20, impulse_t20nonLin = room.T20(impulse_decaycurve, impulse_struct_fmt.fs)
print("impulse_T20=", impulse_t20, ", impulse_T20 nonLin=", impulse_t20nonLin)          # for Debug

# Calculate T30 (-5 ~ -35dB) RT60 = T30 x 2
impulse_t30, impulse_t30nonLin = room.T30(impulse_decaycurve, impulse_struct_fmt.fs)
print("impulse_T30=", impulse_t30, ", impulse_T30 nonLin=", impulse_t30nonLin)          # for Debug


In [ ]:
# Calculate C50, C80, D80

# C50
# ossWav_C50 = room.C50(ossWav_data[:,0], ossWav_fs)
# print("ossWav_C50=", ossWav_C50)         # for Debug
impulse_C50_float32 = room.C50(impulse_data[:,0], impulse_struct_fmt.fs)
print("impulse_C50_float=", impulse_C50_float32)         # for Debug
impulse_C50_int16 = room.C50(np.int16(impulse_data[:,0] * (2**15-1)), impulse_struct_fmt.fs)
print("impulse_C50_int=", impulse_C50_int16)         # for Debug

# C80
impulse_C80 = room.C80(impulse_data[:,0], impulse_struct_fmt.fs)
print("impulse_C80=", impulse_C80)         # for Debug

# D50 
impulse_D50 = room.D50(impulse_data[:,0], impulse_struct_fmt.fs)
print("impulse_D50=", impulse_D50)         # for Debug

